[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/03_Aprendizaje-Por-Refuerzo.ipynb) 👈🏻​ **Pulsar para abrir en Colab​**

# ¿Cómo usar estos notebooks?

Si este es el primer notebook que abres en este repositorio, te recomiendo que antes leas el [Manual de uso de estos notebooks](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/docs/manual-notebooks.md) que he creado para que te familiarices con el proyecto y las distintas rutas que puedes seguir, luego puedes volver aquí y continuar.

En este notebook, vamos a profundizar en el aprendizaje por refuerzo.

Por otra parte, si aún no has revisado el notebook "[00_Empieza-aquí.ipynb](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/00_Empieza-aquí.ipynb)", te sugiero que le eches un vistazo primero para conocer los conceptos básicos. Pero si ya tienes una idea clara de qué es el aprendizaje no supervisado y quieres verlo en acción, ¡estás en el lugar correcto!

# 03. Aprendizaje por Refuerzo

El Aprendizaje por Refuerzo (Reinforcement Learning, RL) es un tipo de Machine Learning donde el modelo (llamado **agente**) aprende a través de la experiencia en un entorno, intentando maximizar una recompensa acumulada a lo largo del tiempo.

## ¿Por qué es importante?

El RL es fundamental en la inteligencia artificial moderna porque nos permite resolver problemas en los que las decisiones se deben tomar secuencialmente y las consecuencias de una acción afectan el futuro. Algunos de los avances más emocionantes en IA han sido gracias al aprendizaje por refuerzo:

- **Inteligencias artificiales campeonas en juegos:** Desde el mítico [AlphaGo](https://en.wikipedia.org/wiki/AlphaGo) que venció a los mejores jugadores de Go, hasta bots que dominan videojuegos como [Dota 2](https://arxiv.org/abs/1912.06680) o [StarCraft](https://deepmind.google/discover/blog/alphastar-grandmaster-level-in-starcraft-ii-using-multi-agent-reinforcement-learning/).
- **Robótica avanzada:** Robots que aprenden a caminar, volar o ensamblar piezas en fábricas sin un manual de instrucciones.
- **Toma de decisiones autónoma:** Algoritmos que optimizan inversiones, rutas de transporte o sistemas energéticos en tiempo real.
- **Ciencia y descubrimiento:** Sistemas que diseñan medicamentos, exploran galaxias o incluso controlan experimentos científicos.

En esencia, el RL se utiliza cuando necesitamos que una máquina aprende a actuar en un entorno complejo y dinámico, en el que no hay una solución clara de antemano.

## ¿Qué vamos a ver?

En este notebook, vamos a explorar el aprendizaje por refuerzo desde sus fundamentos hasta su implementación práctica. El objetivo es que puedas comprender no solo cómo funciona, sino también por qué es tan poderoso.

Al final de este notebook, entenderás:

- **Los fundamentos del RL**, incluyendo conceptos clave como agente, entorno, recompensa y política.
- **Cómo funciona un agente RL**, su interacción con el entorno y cómo aprende para maximizar una recompensa acumulada.
- **Cómo implementar un modelo RL desde cero**, aplicándolo al clásico problema de [CartPole](https://www.gymlibrary.dev/environments/classic_control/cart_pole/).
- **Cómo aplicar RL a diferentes disciplinas**, como negocios, videojuegos, robótica y más.

**¿Listo para empezar a explorar el fascinante mundo del aprendizaje por refuerzo?**

**¡Empecemos!🚀**

# Índice de Contenidos

```{table} Índice
1. Fundamentos del Aprendizaje por Refuerzo
   1.1. Componentes clave del RL  
      1.1.1. Agente, entorno y recompensa  
      1.1.2. Política, función de valor y función Q  
   1.2. Exploración vs. explotación: el equilibrio perfecto  
2. Tabular Q-Learning: Un primer paso práctico
   2.1. Descripción del problema: un agente en una cuadrícula  
   2.2. Configuración del entorno  
   2.3. Implementación del algoritmo Tabular Q-Learning  
      2.3.1. Inicialización de la tabla Q  
      2.3.2. Actualización de valores Q  
   2.4. Visualización del aprendizaje del agente  
   2.5. Reto práctico: Ajustando parámetros y exploración  
3. OpenAI Gym y el problema de CartPole
   3.1. Introducción a OpenAI Gym  
   3.2. Configuración del entorno CartPole  
   3.3. Implementación de Q-Learning en CartPole  
      3.3.1. Entrenamiento del agente  
      3.3.2. Visualización del progreso del agente  
   3.4. Análisis de resultados y reflexiones  
4. Casos de Uso y Reflexión Final
   4.1. Aplicaciones reales del RL  
      4.1.1. Negocios y marketing  
      4.1.2. Robótica  
      4.1.3. Videojuegos  
   4.2. Reflexión sobre las limitaciones y futuro del RL  
   4.3. Conclusión y próximos pasos
```


In [2]:
# Importaciones

import numpy as np

## 1.1. Componentes clave del RL

En este apartado, vamos a desglosar los componentes clave del RL para entender bien cómo podemos aplicarlos a nuestros problemas.

### 1.1.1. Agente, entorno y recompensa

En el corazón del RL, tenemos tres actores principales:

#### 🧑‍🚀 Agente

El agente es el protagonista de nuestra historia. Es la entidad que toma decisiones, explora el mundo y aprende de sus errores. En un videojuego, el agente sería tu personaje controlado. En un robot, el agente sería el sistema que decide cómo moverse.

- **Pregunta clave que se hace el agente**: ¿Qué acción debo tomar ahora?

#### 🌍 Entorno

El entorno es el mundo donde el agente vive y actúa. Define las reglas del juego y las consecuencias de cada acción. Es como el tablero de un juego de mesa: puede ser un simple tablero 2D o un mundo complejo como el universo de Minecraft.

- **Pregunta clave que se hace el agente**: ¿Cómo responde el mundo a mis acciones?

#### 🏆 Recompensa

La recompensa es el "premio" que el agente obtiene después de tomar una acción en el entorno. Es lo que lo motiva a actuar de cierta manera. En un videojuego, la recompensa podría ser un punto extra por recoger una moneda. En un agente de bolsa,la recompensa podría ser la ganancia neta después de una transacción.

- **Pregunta clave que se hace el agente**: ¿Fue buena mi acción?

Las recompensas son lo que determinan cómo el agente ajusta su comportamiento con el tiempo. El agente básicamente vive para maximizar sus recompensas acumuladas, es su propósito vital. Según la situación, podemos encontrar los siguientes escenarios de recompensa:

- **Recompensa positiva**: La acción tomada fue buena. El agente recibe un premio.
  - *Ejemplo*: Un dron recibe +1 por cada segundo que se mantiene volando hacia el objetivo.

- **Recompensa negativa**: La acción tomada fue mala. El agente recibe una penalización.
  - *Ejemplo*: Si el dron choca contra una pared, recibe una penalización de -10.

- **Recompensa nula**: La acción tomada no tiene consecuencias inmediatas. No es ideal, pero tampoco perjudica al agente.
  - *Ejemplo*: El dron empieza a volar en círculos sin progresar hacia el objetivo.

#### Escenarios controlados: Evitar la destruccion del mundo

Como habrás imaginado, no siempre podemos permitir que el agente practique en el mundo real. Imagina un agente aprendiendo a volar un avión a base de prueba y error. ¿Qué podría salir mal? O que un agente practicando cirugías a corazón abierto con pacientes reales. No, gracias🙂‍↔️​.

Para evitar el caos (y salvar el mundo), utilizamos simuladores de mundos controlados. Estos entornos simulan de manera segura la realidad, permitiendo que el agente practique, falle y aprende sin causar daños en el mundo real.

¡Ojo!, esto no sólo aplica a los agentes; los humanos también necesitamos simuladores o entornos controlados para practicar, especialmente en actividades complejas como la conducción o la cirugía. La diferencia es que un agente descontrolado puede tener un potencial de destrucción mucho mayor (y no se detiene a reflexionar sobre sus errores como nosotros).

### 1.1.2. Política, función de valor y función Q

Ahora que tenemos claro quién es quién, pasemos a los conceptos que guían el aprendizaje del agente:

#### 🎯 Política

La política es como el "cerebro" del agente. Define qué acción tomar en cada situación. Puede ser algo tan simple como una tabla de consulta o tan complejo como una red neuronal.

- **Ejemplo:** Si el dron está cerca de una pared, la política podría ser: "Girar a la izquierda para evitarla".

#### 💎 Función de valor

La función de valor le dice al agente qué tan bueno es estar en un estado específico. Es como si el agente tuviera un mapa que indica qué lugares son seguros y cuáles no.

- **Ejemplo:** En el caso del dron, un estado cerca de una pared podría tener un valor bajo (peligroso), mientras que un estado en el centro de la habitación tiene un valor alto (seguro).

#### 🔢 Función Q

La función Q es un nivel más avanzado: no solo evalúa los estados, sino las acciones dentro de esos estados. Es decir, ayuda al agente a decidir cuál acción específica maximizará la recompensa.

- **Ejemplo:** Si el dron está en una esquina, la función Q le diría: "Girando a la derecha tendrás una mejor recompensa que avanzando hacia adelante".


#### Analogía: En busca del tesoro

Piensa en un explorador en un bosque. La política es su instinto para decidir si gira a la izquierda o a la derecha. La función de valor es el mapa que usa para saber qué tan lejos está del tesoro. La función Q combina ambos: "Si tomo este camino, ¿qué tan rápido llegaré al tesoro?"


## 1.2. Exploración vs. explotación: el equilibrio perfecto

En el aprendizaje por refuerzo encontramos un desafío fundamental: **¿cuándo explorar y cuándo explotar?**

Para que un agente aprenda de manera eficiente, necesita encontrar el equilibrio perfecto entre estas dos estrategias:

### Exploración

La exploración consiste en probar nuevas acciones para descubrir más sobre el entorno, incluso si no garantizan una recompensa inmediata. Es como ser un aventurero que se adentra en territorios desconocidos, con la esperanza de encontrar algo valioso.

- **Ventaja**: Puede descubrir estrategias o recompensas que no eran evidentes antes.
- **Desventaja**: Puede tomar decisiones subóptimas en el corto plazo, lo que reduce las recompensas inmediatas.
- **Ejemplo**: Un robot que explora una nueva habitación podría intentar atravesar una puerta que no había detectado antes, descubriendo un camino más corto hacia su objetivo.

### Explotación

La explotación significa usar el conocimiento actual para maximizar las recompensas, eligiendo las acciones que ya sabe que funcionan. Es como ir a tu restaurante favorito y pedir ese plato que nunca te falla. ¿Por qué arriesgarse, verdad?

- **Ventaja**: Asegura recompensas constantes y predecibles.
- **Desventaja**: Limita el descubrimiento de estrategias potencialmente mejores.
- **Ejemplo**: El robot, en lugar de explorar nuevas puertas, siempre usa un camino conocido para llegar a su destino, aunque podría no ser el más eficiente.

### ¿Cómo encontrar el equilibrio?

La clave está en balancear ambas estrategias. Un agente que solo explora nunca aprovecha lo que ha aprendido, mientras que uno que solo explota se queda atascado en soluciones subóptimas. Encontrar este equilibrio es esencial para que el agente no solo aprenda, sino que también logre maximizar su rendimiento.

Esto no es solo un dilema de los agentes; ¡nos pasa a los humanos todo el tiempo! ¿Deberías pedir ese plato que sabes que te encanta o arriesgarte a probar algo nuevo? ¿Ir de vacaciones al mismo lugar de siempre o aventurarte a descubrir un destino desconocido? Tanto para los agentes como para nosotros, el truco está en ser curiosos sin dejar de aprovechar lo que ya sabemos que funciona.

Y tú, ¿eres más explorador o explotador?

# 2. Tabular Q-Learning: Un primer paso práctico

El **Q-Learning** es uno de los algoritmos más básicos y poderosos en el aprendizaje por refuerzo. Es un método basado en tablas que permite a un agente aprender la mejor acción para tomar en cada estado de un entorno.

## 2.1. Descripción del problema: un agente en una cuadrícula

Para este ejemplo, imaginemos que nuestro agente es un explorador en un laberinto de cuadrícula. Su objetivo es encontrar el camino más rápido hacia la salida (o el tesoro escondido) sin chocar con los muros ni caer en trampas. Cada celda del laberinto es un estado, y las acciones posibles son moverse en 4 direcciones: arriba, abajo, izquierda o derecha.

Estas son las reglas del juego:

- **Entorno**: Una cuadrícula 2D (5x5) con una celda de inicio, una meta y varios obstáculos.
- **Estados**: Cada celda de la cuadrícula representa un estado diferente.
- **Acciones**: El agente puede moverse en 4 direcciones: arriba, abajo, izquierda o derecha.
- **Recompensas**:
  - Llegar a la meta: +10 puntos.
  - Chocar contra un muro u obstáculo: -5 puntos.
  - Movimiento normal: -1 punto (penalización mínima para evitar movimientos innecesarios).

**Visualización del problema:**

```plaintext
S: Inicio
G: Meta
X: Obstáculo

    +---+---+---+---+---+
    | S |   |   | X |   |
    +---+---+---+---+---+
    |   |   | X |   |   |
    +---+---+---+---+---+
    |   | X |   |   | G |
    +---+---+---+---+---+
    |   |   |   |   |   |
    +---+---+---+---+---+
    |   |   |   |   |   |
    +---+---+---+---+---+

```

El objetivo del agente en este caso es aprender, a través de prueba y error, cómo moverse desde la celda de inicio (S) hasta la celda de meta (G) mientras minimiza las penalizaciones.

Ahora veamos esto en la práctica: Configurar esta cuadrícula en Python, inicializar la tabla Q, y comenzar a implementar el algoritmo.


## 2.2. Configuración del entorno

Antes de implementar Q-Learning, necesitamos construir nuestro entorno: la cuadrícula 2D donde el agente aprenderá a navegar. Esto incluye definir los estados, las acciones y las reglas del juego.

### Configuración inicial

Vamos a definir el entorno como una cuadrícula 5x5, donde cada celda representa un estado. Utilizaremos Python para estructurar la cuadrícula y asignar las recompensas.

In [3]:
# Tamaño del entorno
grid_size = 5

# Matriz de recompensas
rewards = np.full((grid_size, grid_size), -1)

# Definimos la meta y los obstáculos
rewards[2, 4] = 10
rewards[0, 3] = -5 
rewards[1, 2] = -5
rewards[2, 1] = -5

# Mostrar el entorno inicial
print("Entorno de recompensas:")
print(rewards)


Entorno de recompensas:
[[-1 -1 -1 -5 -1]
 [-1 -1 -5 -1 -1]
 [-1 -5 -1 -1 10]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]


### Definir las acciones

El agente puede moverse en cuatro direcciones: **arriba, abajo, izquierda, derecha**. Vamos a asignar un índice a cada acción para que sea más fácil codificar.

In [4]:
# Acciones posibles
actions = {
    0: "arriba",
    1: "abajo",
    2: "izquierda",
    3: "derecha"
}

### Limitar los movimientos

Para evitar que el agente se salga de los límites de la cuadrícula, definiremos una función que valide sus movimientos.

In [14]:
def validate_movement(pos, action):
    """
    Valida el movimiento del agente en el entorno.
    """
    x, y = pos
    if action == 0 and x > 0:  # Arriba
        return (x - 1, y)
    elif action == 1 and x < grid_size - 1:  # Abajo
        return (x + 1, y)
    elif action == 2 and y > 0:  # Izquierda
        return (x, y - 1)
    elif action == 3 and y < grid_size - 1:  # Derecha
        return (x, y + 1)
    else:
        return pos  # Si el movimiento no es válido, permanece en el mismo lugar

### Probar el entorno

Simulemos un movimiento inicial para verificar que nuestro entorno funciona correctamente.

In [15]:
# Posición inicial del agente
pos_inicial = (0, 0)

# Acción de prueba: moverse hacia la derecha
nueva_pos = validate_movement(pos_inicial, 3)
print(f"El agente se movió de {pos_inicial} a {nueva_pos}")

# Acción inválida: intentar moverse hacia arriba desde el borde superior
nueva_pos = validate_movement((0, 0), 0)
print(f"Movimiento inválido: el agente permanece en {nueva_pos}")


El agente se movió de (0, 0) a (0, 1)
Movimiento inválido: el agente permanece en (0, 0)


## 2.3. Implementación del algoritmo Tabular Q-Learning

El Q-Learning es un algoritmo basado en tablas donde el agente aprende a tomar decisiones optimizadas usando la función Q, que estima la calidad de las acciones en cada estado. Puedes imaginarlo como una hoja de cálculo donde el agente anota qué tan buena es cada acción posible en cada celda del entorno.

### 2.3.1. Inicialización de la tabla Q

La tabla Q es una matriz donde:

- Las filas representan los estados.
- Las columnas representan las acciones.
- Los valores en la tabla indican qué tan buena es una acción en un estado específico.

Primero, crearemos una tabla Q llena de ceros y definiremos algunos parámetros clave.

In [16]:
# Inicializar la tabla Q
q_table = np.zeros((grid_size, grid_size, len(actions)))

# Parámetros del Q-Learning
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 1.0  # Probabilidad inicial de exploración
epsilon_decay = 0.99  # Reducción de epsilon en cada episodio
epsilon_min = 0.1  # Valor mínimo de epsilon

# Mostrar la tabla Q inicial
print("Tabla Q inicial:")
print(q_table)

Tabla Q inicial:
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


> **Nota**: Aquí podemos ver como los valores de la tabla Q valen 0 ahora mismo, esto es porque el agente no tiene experiencia. A medida que aprende, actualizaremos estos valores.

### 2.3.2. Actualización de valores Q

Para que nuestro agente mejore sus decisiones con el tiempo, es fundamental que actualice continuamente su tabla Q. Este proceso de aprendizaje se basa en una regla sencilla: cada vez que el agente toma una decisión, registra la recompensa obtenida. De este modo, construye un historial que le permite identificar las acciones que conducen a los mejores resultados.

El agente utiliza una forma matemática para ajustar los valores de su tabla Q. La idea es combinar:

1. Lo que ya sabe sobre esa acción.
2. Lo que acaba de aprender tras tomar laa acción.

#### Paso a paso del Q-Learning

1. **Elegir una acción**: El agente decide si explorar (acción aleatoria) o explotar (acción con el mejor valor Q conocido).


In [17]:
def choose_action(state, epsilon):
    """
    Elegir acción con epsilon-greedy
    """
    if np.random.rand() < epsilon:
        return np.random.choice(list(actions.keys()))  # Exploración
    else:
        return np.argmax(q_table[state])  # Explotación


En esta función previa, el parámetro `epsilon` es un valor entre `0` y `1` que determina la probabilidad de que el agente explore, un valor más alto, implica una mayor exploración, un valor bajo, implica una mayor explotación.

2. **Actualizar la tabla Q**: El agente observa la recompensa y el nuevo estado, y actualiza el valor Q de la acción tomada.

In [18]:
def update_q(state, action, reward, new_state):
    """
    Actualizar la tabla Q usando la fórmula de Q-Learning
    """
    max_q_new = np.max(q_table[new_state])
    q_table[state][action] += alpha * (reward + gamma * max_q_new - q_table[state][action])

3. **Simulación de episodios**: Un episodio es un intento completo del agente para alcanzar la meta desde el inicio, los episodios se repiten hasta que el agente llega a la meta o fracasa.

Al final de cada episodio, reducimos `𝜖` (epsilon) para que el agente explore menos y explote más.

In [19]:
# Simulación de episodios
episodes = 500
for episode in range(episodes):
    state = (0, 0)  # Inicio
    terminated = False

    while not terminated:
        # Elegir acción
        action = choose_action(state, epsilon)

        # Realizar acción y observar resultado
        new_state = validate_movement(state, action)
        reward = rewards[new_state]

        # Actualizar Q
        update_q(state, action, reward, new_state)

        # Actualizar estado
        state = new_state

        # Verificar si llegamos a la meta
        if state == (2, 4):  # Meta
            terminated = True

    # Reducir epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # Mostrar progreso
    if (episode + 1) % 50 == 0:
        print(f"Episode {episode + 1}/{episodes}: epsilon={epsilon:.2f}")


Episode 50/500: epsilon=0.61
Episode 100/500: epsilon=0.37
Episode 150/500: epsilon=0.22
Episode 200/500: epsilon=0.13
Episode 250/500: epsilon=0.10
Episode 300/500: epsilon=0.10
Episode 350/500: epsilon=0.10
Episode 400/500: epsilon=0.10
Episode 450/500: epsilon=0.10
Episode 500/500: epsilon=0.10
